In [2]:
import os
import json
import uuid
from pathlib import Path
import tiktoken
import pandas as pd
from tqdm import tqdm

# ==== SETTINGS ====
INPUT_FOLDER = r"D:\vc-research\vc-research\batch56_spellcheck"
BATCH_FOLDER = r"D:\vc-research\vc-research\Reese's contributions\reese data\batches\batch56"
OUTPUT_CSV = r"D:\vc-research\vc-research\Reese's contributions\reese data\outputs"
MAX_INPUT_TOKENS = 60_000  # Be safe under 128k limit
MODEL_NAME = "gpt-4o"

# ==== TOKENIZER ====
encoding = tiktoken.encoding_for_model(MODEL_NAME)

def num_tokens(text: str) -> int:
    return len(encoding.encode(text))
# ==== PROMPT GENERATION ====




# Name change flag.
# ==== PROMPT GENERATION ====
def build_prompt(text):
    instruction = """
You are a legal analyst specializing in corporate governance and securities filings, with expertise in parsing incorporation documents. You have been provided with the following text block, drawn from an official corporate charter filing:




Your task:

You must extract a standardized set of legal and financial terms describing the company’s capitalization structure, stockholder rights, and preferred share terms. Return a compact, valid JSON object with the fields described below. If a value is **uncertain, missing, or ambiguous**, return one of the following phrases: if missing from the document state "MISSING", if uncertain or ambiguous state "UNCERTAIN". Do not infer or summarize from context; extract only what is explicitly stated in the document.

---

### LEGAL DEFINITIONS (STRICT):

1. **Company Name**: The exact legal name of the corporation as stated in the title or introductory clause of the certificate. Typically appears in all caps or near the phrase “hereby certifies.” [String]

2. **Document Title**: The formal title of the instrument filed with the Secretary of State. Acceptable examples include:
   - "Certificate of Incorporation"
   - "Amended and Restated Certificate of Incorporation"
   - "Restated Certificate of Incorporation"
   - "amendment #1 to certificate"
   - "amendment two to certificate"
   This must reflect the **actual header** of the document. [String]

3. **incorrect document flag**: Set Inc_Document=1 only if the document type is not a certificate of incorporation or an amendment or restatement of a certificate of incorporation. If the document type is one of the listed types, then set Inc_Document=0.

4. **Filing Date**: The official date the document was filed with the state. This is usually stated in a stamped header or certification block. Convert the date into ISO format (YYYY-MM-DD). [String]

5. **State of Incorporation**: The U.S. state whose corporate law governs the entity, often Delaware. Typically appears in the recitals or the first paragraph. [String]

6. "authorized_common_stock":
The maximum number of common shares the company is legally authorized to issue, as explicitly stated in its certificate of incorporation. This includes all classes or series of common stock (e.g., Class A, Class B). Report the total authorized amount as an integer. If multiple classes are listed separately, sum them.
[Integer or "Uncertain" or "MISSING"]

"authorized_common_stock_d":
Direct quote from the document that supports the authorized common stock figure. This should be a verbatim excerpt (or relevant clause) showing the total number of common shares authorized or allocated to each class, used to justify the extracted number.
[String or "Uncertain" or "MISSING"]

7. "authorized_preferred_stock":
The maximum number of preferred shares the company is authorized to issue, as defined in the certificate of incorporation. This includes the total across all series (e.g., Series A, B, C), regardless of how many have been issued. Report the aggregate authorized amount as an integer.
[Integer or "Uncertain" or "MISSING"]

"authorized_preferred_stock_d":
Direct quote from the document that states the authorized number of preferred shares. This should include the exact language (or segment) used in the certificate, such as “the total number of shares of Preferred Stock that the Corporation shall have authority to issue is...”.
[String or "Uncertain" or "MISSING"]

8. "preferred_stock_series":
A list of each distinct series of preferred stock that is explicitly authorized in the certificate of incorporation or amendment. Include all series mentioned by name (e.g., “Series A,” “Series A-1,” “Series B”). provide the name only in the format "Series X" where X is the series letter or number ex: "Series A" "Series B-2" "Series 3". If no series are specified, return "Uncertain". If the document does not mention any preferred stock, return "MISSING". 
[List of Strings or "Uncertain" or "MISSING"]

"preferred_stock_series_d":
Direct quote(s) from the document listing the series of preferred stock. This should include the relevant passage(s) where the series are named, such as “The Corporation shall have authority to issue 5,000,000 shares of Series A Preferred Stock…”
[String or "Uncertain" or "MISSING"]

9. "total_shares_authorized":
The combined total number of shares the company is authorized to issue across all classes and series, including both common and preferred stock. This reflects the aggregate share authorization as stated in the certificate. 
[Integer or "Uncertain" or "MISSING"]

"total_shares_authorized_d":
Direct quote from the document that provides or supports the total authorized share count. This may appear as a summary statement (e.g., “The total number of shares of all classes which the corporation is authorized to issue is...”) or may be inferred by summing individual class authorizations when such a line is absent (e.g. if there are 1 million common stock shares authorized and 2 million preferred stock shares authorized, make total shares authorized 3 million).
[String or "Uncertain" or "MISSING"]

10. **Multiple Common Stock Classes**:
   - Return 1 if the certificate authorizes more than one class of common stock (e.g., Class A and Class B). Often the different classes of stock will have different voting rights per share, such as stock A will have one vote per share and stock B will have 5 votes per share, or in some cases there may be 0 votes per share.
   - Return 0 if there is only one class of common stock authorized.
   - If 1, also extract a dictionary of the **voting rights** assigned to each class, such as:
     {"Class A": "1 vote per share", "Class B": "10 votes per share"}
   - If 1, also extract a dictionary of the number of issued shares of each class, as in:
     {"Class A": "10,000,000", "Class B": "100,000"}
   - This reflects differences in shareholder control and is central to dual-class structures.
   - If voting rights are not clearly stated, set to "MISSING", if they are ambiguous, set to "Uncertain".

11. **Preferred Stock Terms**: For each listed preferred stock series, extract the following detailed fields:    
    - **Shares authorized**:  The total number of authorized shares of **this series**. [Integer or "Uncertain"]
    - **Liquidation Preference**:  
      The dollar amount per share a preferred shareholder is entitled to receive before common shareholders upon liquidation, dissolution, or winding up of the company. 
      - If defined as equal to the “Original Issue Price,” extract that numeric value and flag that as the basis. Alternatively it may be benchmarked against "issue price" or a similar time.
      - Example: "Amount": 1.00, "Based On": "Original Issue Price"
    - **liquidation multiple**:
	The multiplier applied to the original investment during a liquidation 
event (e.g., 1, 2x). ""N/A"" indicates this information is not provided."
    - **Conversion Price**:  
      The price per share at which preferred stock may be converted into common stock. This is often fixed and based on the original issuance terms. If a benchmark is provided, state that in the field: **Conversion Price_d**
    - **Dividend Rate (Annual)**:  
      The annual dividend expressed as a percentage of the original issue price or par value, usually stated as “8% per annum.” or “$.45 per annum.”[return as string with units]
    - **Cumulative Dividends**:  
      Cumulative dividends accrue if not declared or paid in a given year. Return "1" if dividends are cumulative, otherwise "0".
    - **Participation Rights**:  
      Whether the preferred shares are entitled to participate in additional common distributions after their liquidation preference amount (and any accrued dividends) have been paid beyond their fixed return:

        - "P" = Full Participation; 
Full participation refers to a liquidation preference structure where preferred shareholders first receive their liquidation preference (typically their original investment amount or a multiple thereof), and then also share pro rata in any remaining proceeds with common stockholders—as if they had converted to common stock. ex: “The holders of Series A Preferred shall first receive an amount equal to 1x the Original Issue Price, and thereafter shall participate with the common stockholders on an as-converted basis.” “Preferred Stock shall be entitled to full participation with Common Stock after payment of its liquidation preference.”

        - "C" = Capped Participation; 
Capped participation in venture capital refers to a provision where preferred shareholders receive their liquidation preference plus a share in remaining proceeds up to a specified cap, after which they stop participating. This limits their upside after a certain return multiple. ex: “Participating preferred stock shall receive a pro rata share of remaining proceeds until total proceeds equal 3x the Original Issue Price.” “Participation shall be capped at 2 times the liquidation preference.” “Holders shall participate with common stockholders on an as-converted basis, subject to a 4x cap.”
        - "N" = Non-Participating


    - ** "dividend_priority_order":
If dividends are to be distributed in a specific sequence across classes or series, summarize that hierarchical order. This may include preferred series receiving dividends before common stock or certain series having non-participating rights, it may be the case that some classes are paid simultaneously, or are equal in the hierarchy, or there may be no hierarchy. (e.g., “Series A then Common”) ex:“No dividends shall be declared or paid on the Common Stock unless and until the holders of Series A Preferred Stock shall have received dividends at the annual rate of 8% of the Original Issue Price.” “Dividends shall be paid in the following order of priority: first, to the holders of Series A Preferred Stock, until they have received their stated dividend in full; second, to the holders of Series B Preferred Stock…” ;  format in the following style, for any prioritized dividends write "A > B" and for any equal ranked dividends write "A=B" for the series involved. This should include all stock series in the ranking.
[String or "Uncertain" or "MISSING"]

"dividend_priority_order_d":
Direct quote from the document that describes how dividends are prioritized among stockholders. Look for language such as “dividends shall be paid first to holders of Series A Preferred Stock…” or “no dividends shall be paid on Common unless and until the Series B Preferred Stock receives…”
[String or "Uncertain" or "MISSING"]

    - **Antidilution Protection**:  
      A contractual adjustment provision that protects preferred shareholders from dilution during future equity offerings. If there is an additional round of qualifying financing, it automatically adjusts the conversion price such that the preferred shareholder will receive more common shares. Describe the formula if given (e.g., “broad-based weighted average” or “full ratchet”). 0 if no antidilution protection, 1 if there is, and **Antidilution Type**:
	N = None, 
	R = Rachet, defined as adjusts the preferred stock’s conversion price down to the price of the new issuance, regardless of how many shares are issued, giving stronger protection against dilution. ex:
“In the event of an issuance below the original issue price, the conversion price shall be adjusted to match the lower price.” “Full ratchet antidilution shall apply for any subsequent down-round financings.”
	W = Weighted Average, defined as: adjusts the preferred stock's conversion price when new shares are issued at a lower price, using a formula that accounts for both the price and quantity of the new shares—mitigating but not fully offsetting dilution. ex: “The conversion price shall be adjusted on a broad-based weighted average basis in the event of a dilutive issuance.” “Antidilution protection shall apply using a broad-based weighted average formula, including options and warrants.”
	O = Other
    - **Antidilution Protection_d**:
 	Describe the formula of the antidilution protection if given (e.g., “broad-based weighted average” or “full ratchet”) 
    - **Redemption Rights**:  
      Whether the issuer (or shareholder) has the right to repurchase shares at a future time. [1 if redemption rights, 0 if none] Summarize the redemption triggers or mechanics if present.
    - **Voting Rights**:  
      Any voting rights granted to this preferred series. this may be an increase to voting amount or a decrease to voting power in specific cases, or mandatory votes for certain conditions. ex: “Each share of Series A Preferred Stock shall entitle the holder to ten (10) votes per share on all matters submitted to a vote of the stockholders.” “Holders of Series C Preferred shall have a class vote on any amendment to the Certificate of Incorporation that adversely affects their rights, preferences, or privileges.”
    -**special class protections**:  rights that require the separate consent of a specific class or series of stock—typically preferred stock—before the company can take certain major corporate actions (e.g., issuing senior securities, changing rights, selling the company, etc.). These rights protect investors from actions that could dilute or disadvantage their position. ex:“So long as any shares of Series A Preferred Stock remain outstanding, the Company shall not, without the consent of the holders of at least a majority of the Series A Preferred Stock, authorize or issue any equity security having rights senior to or on parity with the Series A Preferred Stock.” “A merger, consolidation, sale of all or substantially all of the assets of the Company, or any other Deemed Liquidation Event shall require the approval of a majority of the outstanding Series C Preferred Stock, voting as a separate class.”
    -**special class protections_d**: quote the passage that any special class protections are mentioned directly from the text provided, if no protections were given set this "special class protections_d = 0"



12. **liquidation preference flag**: set liq_pref_flag=1 only if the ranking or order in which the liquidity preference is paid out is referenced in the document. Otherwise set liq_pref_flag=0.
[0 or 1]


13. "liquidation_priority_order":
A list showing the explicit order in which stockholders are paid in the event of a liquidation, dissolution, or winding up of the company. This typically reflects which preferred series have priority over others, and whether common stockholders receive proceeds only after preferred holders are fully paid. Format in the following style, for any prioritized dividends write "A > B" and for any equal ranked dividends write "A=B" for the series involved. This should include all stock series in the ranking.
[String or "Uncertain" or "MISSING"]
[List or "Uncertain" or "MISSING"]

"liquidation_priority_order_d":
Direct quote(s) from the document that describe the relative liquidation rights of each series or class. Look for phrases such as “prior and in preference to,” “junior to,” or “after payment to holders of Series A Preferred Stock…”
[String or "Uncertain" or "MISSING"]

14. "conversion_rights":
A summary of the rights of holders or the company to convert preferred shares into common stock, including whether the conversion is optional, automatic, or conditional (e.g., upon IPO, vote of majority holders, or other events). Include relevant terms such as conversion ratios or procedures if specified.
[String or "Uncertain" or "MISSING"]

"conversion_rights_d":
Direct quote from the document that outlines the conversion rights, such as “Each share of Series A Preferred Stock shall be convertible, at the option of the holder, at any time…” or “shall automatically convert upon the closing of a Qualified IPO…”
[String or "Uncertain" or "MISSING"]

15. "automatic_conversion_triggers":
Specific events or conditions that cause mandatory (automatic) conversion of preferred stock into common stock. Common triggers include a firm-commitment IPO above a defined size (e.g., $50M), qualified public offering, or majority vote of preferred holders. Summarize the criteria clearly.
[String or "Uncertain" or "MISSING"]

"automatic_conversion_triggers_d":
Direct quote from the document specifying the conditions that trigger automatic conversion, such as “Each share of Series B Preferred Stock shall automatically convert into Common Stock upon the closing of a firm-commitment underwritten public offering with gross proceeds of at least $50,000,000…”
[String or "Uncertain" or "MISSING"]




16. "merger_indicator":
Set to 1 if the document specifies a merger between the company and another entity, or if it states that the company is a successor to another entity. Set to 0 if no such information is present. This indicates whether the company has undergone a merger or is part of a larger corporate structure.
[0 or 1]
"merger_indicator_d":
Direct quote from the document that indicates a merger or succession, such as “The Corporation is the successor to XYZ Corp. by merger” or “pursuant to the merger agreement dated…” If no merger is mentioned, set this to "0".
[String or "Uncertain" or "MISSING"]

17. **Name Change Flag**:  
    Set to 1 if the document indicates that the company has changed its name from a previous name, or if it states that the company is a successor to another entity with a different name.
    Set to 0 if no such information is present. [0 or 1]


18. **Notable Features Summary**:  
    A one-sentence summary of any **unusual or legally significant provisions**, such as “Series A holders must approve any future IPO under $75 million.” This should highlight unique investor rights or governance features.
special class protections
---

### Return your output as valid compact JSON ONLY:
{
  "company_name": "...",
  "document_title": "...",
  "inc_document": 0,
  "filing_date": "...",
  "state_of_incorporation": "...",
  "authorized_common_stock": 10000000,
  "authorized_common_stock_d": "Uncertain",
  "authorized_preferred_stock": 5000000,
  "authorized_preferred_stock_d": "Uncertain",
  "preferred_stock_series": ["Series A", "Series B"],
  "preferred_stock_series_d": "Uncertain",
  "total_shares_authorized": 15000000,
  "total_shares_authorized_d": "Uncertain",
  "multiple_common_stock_classes": 1,
  "common_voting_rights": [
    "Class A": "1 vote per share",
    "Class B": "10 votes per share"
  ],
  "common_issued_shares": "Uncertain",
  "preferred_stock_terms": {
    "Series A": [
      "shares_authorized": "Uncertain",
      "liquidation_preference": [
        "amount": 1.00,
        "based_on": "Original Issue Price"
      ],
      "liquidation_multiple": "N/A",
      "conversion_price": 1.00,
      "conversion_price_d": "Uncertain",
      "dividend_rate_annual": "8% per annum",
      "cumulative_dividends": "1",
      "participation_rights": "P",
      "dividend_priority_order": "Uncertain",
      "dividend_priority_order_d": "Uncertain",
      "antidilution_protection": 1,
      "antidilution_type": "W",
      "antidilution_protection_d": "Broad-based weighted average",
      "redemption_rights": [
        "present": 1,
        "description": "At holder’s option after 5 years"]
      ,
      "voting_rights": "1 vote per share",
      "special_class_protections": 1,
      "special_class_protections_d": "Uncertain"
    ]
  },
  "liq_pref_flag": 1,
  "liquidation_priority_order": "A>B=C",
  "liquidation_priority_order_d": "Uncertain",
   "conversion_rights": "Optional at any time by holder",
   "conversion_rights_d": "Uncertain",
  "automatic_conversion_triggers": "Qualified IPO above $50 million",
  "automatic_conversion_triggers_d": "Uncertain",
  "merger_indicator": "0",
  "merger_indicator_d": "MISSING",
  "Name_change_flag": "1",
  "notable_features_summary": "Series A holders have blocking rights on change of control."
}

Please analyze the following legal document and return only the JSON result.
"""

    prompt = f"{instruction.strip()}\n\nLegal Text:\n{text.strip()}"
    return prompt



def create_batches(groupid):
    Path(BATCH_FOLDER).mkdir(exist_ok=True, parents=True)

    batch = []
    batch_id = 0
    token_total = 0

    for fname in tqdm(os.listdir(INPUT_FOLDER)):
        #print(fname)
        if not fname.endswith(".txt"):
            continue

        with open(os.path.join(INPUT_FOLDER, fname), 'r', encoding='utf-8') as f:
            text = f.read()

        prompt = build_prompt(text)
        tokens = num_tokens(prompt)

        # Split into new batch if over token limit
        if token_total + tokens > MAX_INPUT_TOKENS:
            # Save current batch
            batch_path = os.path.join(BATCH_FOLDER, f"batch_{batch_id}_{groupid}.jsonl")
            with open(batch_path, 'w', encoding='utf-8') as f:
                for item in batch:
                    f.write(json.dumps(item) + "\n")
            print(f"🔄 Saved batch {batch_id} with {len(batch)} items and {token_total:,} tokens")

            # Start new batch
            batch_id += 1
            batch = []
            token_total = 0

        # Add prompt to batch
        task_id = f"{Path(fname).stem}_{uuid.uuid4().hex[:8]}"
        batch.append({
            "custom_id": task_id,
            "method": "POST",
            "url": "/v1/chat/completions",
            "body": {
                "model": MODEL_NAME,
                "temperature": 0.0,
                "response_format": {"type": "json_object"},  # ✅ FIXED
                "messages": [
                    {"role": "system", "content": "You are a legal analyst trained to extract structured fields from incorporation documents."},
                    {"role": "user", "content": prompt}
                ]
            }
        })

        token_total += tokens

    # Final batch
    if batch:
        batch_path = os.path.join(BATCH_FOLDER, f"batch_{batch_id}_{groupid}.jsonl")
        with open(batch_path, 'w', encoding='utf-8') as f:
            for item in batch:
                f.write(json.dumps(item) + "\n")
        print(f"✅ Final batch {batch_id} saved with {len(batch)} items and {token_total:,} tokens")



# ==== CSV Parsing from Responses ====
def parse_outputs(response_folder=r"outputs"):
    records = []
    for fname in os.listdir(response_folder):
        if not fname.endswith(".jsonl"):
            continue

        with open(os.path.join(response_folder, fname), 'r', encoding='utf-8') as f:
            for line in f:
                obj = json.loads(line)
                if 'response' in obj and 'message' in obj['response'] and 'content' in obj['response']['message']:
                    try:
                        content = obj['response']['message']['content']
                        parsed = json.loads(content)
                        parsed["task_id"] = obj.get("custom_id")
                        records.append(parsed)
                    except Exception as e:
                        print(f"❌ Error parsing output for {obj.get('custom_id')}: {e}")

    df = pd.DataFrame(records)
    df.to_csv(OUTPUT_CSV, index=False)
    print(f"✅ Extracted data saved to: {OUTPUT_CSV}")

# ==== MAIN ====
if __name__ == "__main__":
    create_batches(groupid="G56")
    # parse_outputs()  # Run this **after** you download the batch results


ModuleNotFoundError: No module named 'tiktoken'

In [1]:
import openai
import os
import time
import apikey

# Set your API key
openai.api_key = apikey.get_api_key()  # or hardcode directly if needed

# Base folder for your batch files
batch_folder = r"D:\vc-research\vc-research\Reese's contributions\reese data\batches\batch56"
log_file_path = os.path.join(batch_folder, "batch_upload_log.txt")

# Open the log file in append mode
with open(log_file_path, "a", encoding="utf-8") as log_file:

    # Loop through batches 0 to 25
    for i in range(3900,5200):
        batch_file_path = os.path.join(batch_folder, f"batch_{i}_G56.jsonl")

        if not os.path.exists(batch_file_path):
            print(f"⚠️ File not found: {batch_file_path}, skipping.")
            continue

        try:
            # Upload the file
            with open(batch_file_path, "rb") as f:
                uploaded_file = openai.files.create(file=f, purpose="batch")
                file_id = uploaded_file.id

            print(f"[Batch {i}] ✅ Uploaded file. File ID: {file_id}")

            # Create the batch job
            batch = openai.batches.create(
                input_file_id=file_id,
                endpoint="/v1/chat/completions",
                completion_window="24h",  # or "48h"
            )

            print(f"[Batch {i}] ✅ Batch created. Batch ID: {batch.id}")
            print(f"[Batch {i}] 🕒 Status: {batch.status}")
            print(f"[Batch {i}] 🔗 Monitor: https://platform.openai.com/batch/{batch.id}\n")

            # Write to log
            log_file.write(f"Batch {i} | File ID: {file_id} | Batch ID: {batch.id} | Status: {batch.status}\n")
            log_file.flush()

        except Exception as e:
            print(f"❌ Error processing Batch {i}: {e}")
            log_file.write(f"Batch {i} | ERROR: {e}\n")
            log_file.flush()

        # Pause to avoid hitting API rate limits
        time.sleep(75)  # adjust if needed


[Batch 3900] ✅ Uploaded file. File ID: file-AVPXzgTMa3b54cEYdF3KbJ
[Batch 3900] ✅ Batch created. Batch ID: batch_69121f5c8e288190b62fc16658eecc98
[Batch 3900] 🕒 Status: validating
[Batch 3900] 🔗 Monitor: https://platform.openai.com/batch/batch_69121f5c8e288190b62fc16658eecc98

[Batch 3901] ✅ Uploaded file. File ID: file-Pfdh4P6Hx7SB7TRJd9u7iT
[Batch 3901] ✅ Batch created. Batch ID: batch_69121fae77d08190be461384f135c148
[Batch 3901] 🕒 Status: validating
[Batch 3901] 🔗 Monitor: https://platform.openai.com/batch/batch_69121fae77d08190be461384f135c148

[Batch 3902] ✅ Uploaded file. File ID: file-3BEWgbNt9HPzzPq7cU5g6E
[Batch 3902] ✅ Batch created. Batch ID: batch_69121ffb73148190951d740762e76cce
[Batch 3902] 🕒 Status: validating
[Batch 3902] 🔗 Monitor: https://platform.openai.com/batch/batch_69121ffb73148190951d740762e76cce

[Batch 3903] ✅ Uploaded file. File ID: file-HDZsMppdoz25UpZdt5zM8V
[Batch 3903] ✅ Batch created. Batch ID: batch_6912204c5abc8190ba5e7879cb7a706c
[Batch 3903] 🕒 Statu

In [7]:
import openai
import os
import apikey
# Set your API key
openai.api_key = apikey.get_api_key()  # or hardcode: "sk-..."

# File to upload
batch_file_path = r"D:\vc-research\vc-research\Reese's contributions\reese data\batches\batch2\batch_103_G2.jsonl"

# Upload the file to OpenAI
with open(batch_file_path, "rb") as f:
    uploaded_file = openai.files.create(file=f, purpose="batch")
    file_id = uploaded_file.id

print(f"✅ Uploaded file. File ID: {file_id}")

# Create the batch job
batch = openai.batches.create(
    input_file_id=file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",  # Use "24h" or "48h"
)

print(f"✅ Batch created. Batch ID: {batch.id}")
print(f"🕒 Status: {batch.status}")
print(f"🔗 You can monitor the batch here: https://platform.openai.com/batch/{batch.id}")

✅ Uploaded file. File ID: file-74J7hxSqE5oAgGQyWnfyQJ
✅ Batch created. Batch ID: batch_68adb5eadbfc819080354143c72d0d48
🕒 Status: validating
🔗 You can monitor the batch here: https://platform.openai.com/batch/batch_68adb5eadbfc819080354143c72d0d48


In [3]:
import openai
import os
import time
import requests

# Replace with your batch ID
BATCH_ID = "batch_687f96d574a481909ecc029516e9d122"  # Paste your actual ID here

# Output path
OUTPUT_PATH = r"D:\vc-research\reese data\outputs\batch_0_output.jsonl"
os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)

 # or hardcode

def download_batch_output(batch_id, output_path):
    while True:
        batch = openai.batches.retrieve(batch_id)
        status = batch.status
        print(f"Status: {status}")

        if status in ("completed", "failed", "cancelled"):
            break

        time.sleep(10)  # wait before polling again

    if status != "completed":
        print(f"❌ Batch failed or was cancelled.")
        return

    # Get output file URL
    output_url = batch.output_file_urls["response_file_url"]
    print(f"🔽 Downloading from {output_url}")

    # Download the output
    response = requests.get(output_url)
    if response.status_code == 200:
        with open(output_path, 'wb') as f:
            f.write(response.content)
        print(f"✅ Output saved to {output_path}")
    else:
        print(f"❌ Failed to download: HTTP {response.status_code}")

if __name__ == "__main__":
    download_batch_output(BATCH_ID, OUTPUT_PATH)


Status: cancelled
❌ Batch failed or was cancelled.


In [5]:
import openai
import os
from datetime import datetime
from tqdm import tqdm
import apikey
# Set your API key
openai.api_key = apikey.get_api_key()  # or hardcode: "sk-..."


# === SETTINGS ===

BATCH_LIMIT = 100  # How many recent batches to list

# === CLEANUP SCRIPT ===
def human_time(unix_ts):
    return datetime.fromtimestamp(unix_ts).strftime("%Y-%m-%d %H:%M")

def clean_batches():
    print(f"🔍 Retrieving your latest {BATCH_LIMIT} batches...")
    batches = list(openai.batches.list(limit=BATCH_LIMIT))

    for b in tqdm(batches):
        bid = b.id
        status = b.status
        created = human_time(b.created_at)
        description = f"[{status.upper():10}] {bid} ({created})"

        if status in ("validating", "queued", "in_progress"):
            try:
                openai.batches.cancel(bid)
                print(f"🚫 Canceled: {description}")
            except Exception as e:
                print(f"❌ Error canceling {bid}: {e}")
        
        else:
            print(f"✅ Skipped (status={status}): {description}")

if __name__ == "__main__":
    clean_batches()


🔍 Retrieving your latest 100 batches...


100%|██████████| 2409/2409 [00:00<00:00, 59930.71it/s]

✅ Skipped (status=completed): [COMPLETED ] batch_68effab8cbfc8190b5729c6c68b64773 (2025-10-15 14:49)
✅ Skipped (status=completed): [COMPLETED ] batch_68effa6ce4a88190ab1ff5bdee7c572d (2025-10-15 14:47)
✅ Skipped (status=completed): [COMPLETED ] batch_68effa2076908190b0b295541c60c9b6 (2025-10-15 14:46)
✅ Skipped (status=completed): [COMPLETED ] batch_68eff9d3b81c81908abd36f6c0f95c0f (2025-10-15 14:45)
✅ Skipped (status=completed): [COMPLETED ] batch_68eff9871768819086121ce6a61e6e14 (2025-10-15 14:44)
✅ Skipped (status=completed): [COMPLETED ] batch_68eff93af1808190a3c7d9c2c2cf4d77 (2025-10-15 14:42)
✅ Skipped (status=completed): [COMPLETED ] batch_68eff8eea8748190b1aef547e8f9faa6 (2025-10-15 14:41)
✅ Skipped (status=completed): [COMPLETED ] batch_68eff8a1abf481909c8b1cb32c42548b (2025-10-15 14:40)
✅ Skipped (status=completed): [COMPLETED ] batch_68eff855695c81909a588474f7a5db74 (2025-10-15 14:39)
✅ Skipped (status=completed): [COMPLETED ] batch_68eff808f63c8190a0c442d46ee8a5e3 (2025-10-

In [6]:
import os
import re
import time
import json
import apikey
import openai

# ===== Setup =====
openai.api_key = apikey.get_api_key()

# Adjust these paths as needed
batch_folder = r"D:\vc-research\reese data\batches\batch1"
log_file_path = os.path.join(batch_folder, "batch_upload_log.txt")

# Where to save downloaded results
out_dir = os.path.join(batch_folder, "downloads")
os.makedirs(out_dir, exist_ok=True)

# Pause between API calls (seconds)
PAUSE_SEC = 3


def parse_batch_ids_from_log(log_path):
    """Extract Batch IDs from the upload log."""
    batch_ids = []
    if not os.path.exists(log_path):
        print(f"⚠️ Log not found: {log_path}")
        return batch_ids

    pat = re.compile(r"Batch ID:\s*(batch_[\w\-]+)")
    with open(log_path, "r", encoding="utf-8") as f:
        for line in f:
            m = pat.search(line)
            if m:
                batch_ids.append(m.group(1))
    # Deduplicate, preserve order
    seen = set()
    uniq = []
    for b in batch_ids:
        if b not in seen:
            uniq.append(b)
            seen.add(b)
    return uniq


def _save_file_content(file_id, dest_path):
    """
    Download and save file content for a file_id using Files API.
    Handles text or bytes responses depending on SDK version.
    """
    resp = openai.files.content(file_id)
    # Try .content (bytes) first; fall back to .text if available
    content = getattr(resp, "content", None)
    if content is None:
        # Some SDKs return a requests-like Response with .text
        text = getattr(resp, "text", None)
        if text is None:
            # Fallback: treat as str
            text = str(resp)
        with open(dest_path, "w", encoding="utf-8") as f:
            f.write(text)
    else:
        # content is bytes
        with open(dest_path, "wb") as f:
            f.write(content)


def download_batch(batch_id):
    """Retrieve batch status, download output and error files if available."""
    try:
        batch = openai.batches.retrieve(batch_id)
    except Exception as e:
        print(f"❌ Could not retrieve {batch_id}: {e}")
        return

    status = getattr(batch, "status", "unknown")
    output_file_id = getattr(batch, "output_file_id", None)
    error_file_id = getattr(batch, "error_file_id", None)

    print(f"🔎 {batch_id} | status={status} | output={output_file_id} | errors={error_file_id}")

    # Only completed/expired batches have downloadable files
    if status not in ("completed", "expired"):
        print(f"⏭️  Skipping {batch_id}: not ready (status={status}).")
        return

    # Prepare filenames
    base = batch_id.replace("batch_", "")
    out_path = os.path.join(out_dir, f"{base}_output.jsonl")
    err_path = os.path.join(out_dir, f"{base}_errors.jsonl")

    # Download output file (if present)
    if output_file_id:
        if os.path.exists(out_path):
            print(f"📝 Output already exists: {out_path} (skipping download)")
        else:
            try:
                _save_file_content(output_file_id, out_path)
                print(f"✅ Saved output: {out_path}")
            except Exception as e:
                print(f"❌ Failed to save output for {batch_id}: {e}")
        time.sleep(PAUSE_SEC)

    # Download error file (if present)
    if error_file_id:
        if os.path.exists(err_path):
            print(f"📝 Errors already exist: {err_path} (skipping download)")
        else:
            try:
                _save_file_content(error_file_id, err_path)
                print(f"✅ Saved errors: {err_path}")
            except Exception as e:
                print(f"❌ Failed to save errors for {batch_id}: {e}")
        time.sleep(PAUSE_SEC)


if __name__ == "__main__":
    batch_ids = parse_batch_ids_from_log(log_file_path)
    if not batch_ids:
        print("⚠️ No batch IDs found in the log. If needed, add them manually or list via the API.")
    else:
        print(f"Found {len(batch_ids)} batch IDs in log.")
        for bid in batch_ids:
            download_batch(bid)


⚠️ Log not found: D:\vc-research\reese data\batches\batch1\batch_upload_log.txt
⚠️ No batch IDs found in the log. If needed, add them manually or list via the API.
